# Data Handling of Graphs

A graph is used to model pairwise relations (edges) between objects (nodes). A single graph in PyTorch Geometric is described by an instance of torch_geometric.data.Data, which holds the following attributes by default:

그래프는 pairwise 관계(edges)를 사용한다. single graph는 '''torch_geometric.data.Data''' 에 설명되어있다.

- data.x
    - Node feature matrix with shape [num_nodes, num_node_features]
    - [노드 갯수, 특성의 차원]

- data.edge_index
    - Graph connectivity in COO format with shape [2, num_edges] and type torch.long
    - 어떤 노드와 어떤 노드가 연결되어있는지 나타내는 특성인듯

- data.edge_attr
    - Edge feature matrix with shape [num_edges, num_edge_features]
    - 엣지의 피쳐

- data.y
    - Target to train against (may have arbitrary shape), e.g., node-level targets of shape [num_nodes, *] or graph-level targets of shape [1, *]
    - 학습시킬 것의 형태를 말하는 듯
    
- data.pos
    - Node position matrix with shape [num_nodes, num_dimensions]
    - 실제 위치를 나타내는듯?

None of these attributes is required. In fact, the Data object is not even restricted to these attributes. We can, e.g., extend it by data.face to save the connectivity of triangles from a 3D mesh in a tensor with shape [3, num_faces] and type torch.long.

이 특성들이 필요한 것은 아니다. 사실, Data 객체는 이 특성들에 제한되지않는다. 우리는 data.face를 통해서 [3, num_faces]의 크기를 가진 텐서의 3D mesh로부터 삼각형들의 연결성을 저장함으로써 그것을 확장시킬 수 있다(?)

We show a simple example of an unweighted and undirected graph with three nodes and four edges. Each node contains exactly one feature:

- unweighted
- undirected
- one feature

In [1]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float) #노드의 특성 여기선 1차원이니까 실수 하나


![graph](img/graph.svg)

In [2]:
data = Data(x=x, edge_index=edge_index)
data

Data(edge_index=[2, 4], x=[3, 1])

Note that '''edge_index''', i.e. the tensor defining the source and target nodes of all edges, is NOT a list of index tuples. If you want to write your indices this way, you should transpose and call '''contiguous''' on it before passing them to the data constructor:

In [3]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

In [4]:
data = Data(x=x, edge_index=edge_index.t().contiguous())
data #위와 같은 방법인데, edge_index를 다른 형태로 나타낸듯?

Data(edge_index=[2, 4], x=[3, 1])

Besides of being a plain old python object, '''torch_geometric.data.Data''' provides a number of utility functions, e.g.:

In [5]:
print(data.keys)

['x', 'edge_index']


In [6]:
#노드의 피쳐들을 나타냄.
print(data['x'])

tensor([[-1.],
        [ 0.],
        [ 1.]])


In [8]:
for key, item in data:
    print("{} found in data".format(key))

edge_index found in data
x found in data


In [9]:
'edge_attr' in data

False

In [10]:
data.num_nodes

3

In [11]:
#단방향을 하나로 치고, 2*2 = 4
data.num_edges

4

In [12]:
data.num_node_features

1

In [13]:
data.contains_isolated_nodes()

False

In [14]:
data.contains_self_loops()

False

In [15]:
#모두 양방향이니까
data.is_directed()

False

In [16]:
device = torch.device('cuda')
data = data.to(device)

You can find a complete list of all methods at '''torch_geometric.data.Data'''



# Common Benchmark Datasets 

PyTorch Geometric contains a large number of common benchmark datasets, e.g. all Planetoid datasets (Cora, Citeseer, Pubmed), all graph classification datasets from http://graphkernels.cs.tu-dortmund.de/ and their cleaned versions, the QM7 and QM9 dataset, and a handful of 3D mesh/point cloud datasets like FAUST, ModelNet10/40 and ShapeNet.

Initializing a dataset is straightforward. An initialization of a dataset will automatically download its raw files and process them to the previously described Data format. E.g., to load the ENZYMES dataset (consisting of 600 graphs within 6 classes), type:

- raw file을 받아서 프로세싱 하는 것은 Data format칸에서 설명하고 여기에서는 간단한 것만 해보겠다.

In [7]:
from torch_geometric.datasets import TUDataset

In [8]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [9]:
print(len(dataset)) #600
print(dataset.num_classes) #6
print(dataset.num_node_features) #3

600
6
3


In [10]:
#그래프 600개중에 첫 번째 그래프
data = dataset[0]
data

Data(edge_index=[2, 168], x=[37, 3], y=[1])

In [11]:
data.is_undirected()

True

We can see that the first graph in the dataset contains 37 nodes, each one having 3 features. There are 168/2 = 84 undirected edges and the graph is assigned to exactly one class. In addition, the data object is holding exactly one graph-level target.

- 37개의 노드들이랑 3만큼의 피쳐를 가진다.
- 168/2 만큼의 undirected edge를 가지고 
- 그래프는 하나의 클래스를 가진다.
- 여기서 학습시킬 것은 그래프 level의 target이다.

We can even use slices, long or byte tensors to split the dataset. E.g., to create a 90/10 train/test split, type:

If you are unsure whether the dataset is already shuffled before you split, you can randomly permutate it by running:

- 섞고싶을때 이걸 써라
- 주석 친 두가지 모두 섞는 방법들임.

In [22]:
#dataset = dataset.shuffle() 

#perm = torch.randperm(len(dataset))
#dataset = dataset[perm]

train_dataset = dataset[:540] #ENZYMES(540)
test_dataset = dataset[540:] #ENZYMES(60)

Let’s try another one! Let’s download '''Cora''', the standard benchmark dataset for semi-supervised graph node classification:

In [12]:
from torch_geometric.datasets import Planetoid

In [13]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
dataset

Cora()

In [14]:
len(dataset) #1
dataset.num_classes #7
dataset.num_node_features #1433

1433

Here, the dataset contains only a single, undirected citation graph:

In [15]:
data = dataset[0]
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [33]:
data.is_undirected()

True

This time, the Data objects holds a label for each node, and additional attributes: train_mask, val_mask and test_mask:

- train_mask
    - denotes against which nodes to train (140 nodes)
    
-  val_mask
    - denotes which nodes to use for validation, e.g., to perform early stopping (500 nodes)
    
- test_mask
    - denotes against which nodes to test (1000 nodes)

In [37]:
data.train_mask.sum().item()

140

In [35]:
data.val_mask.sum().item()

500

In [36]:
data.test_mask.sum().item()

1000

# Mini-batches

Neural networks are usually trained in a batch-wise fashion. PyTorch Geometric achieves parallelization over a mini-batch by creating sparse block diagonal adjacency matrices (defined by edge_index and edge_attr) and concatenating feature and target matrices in the node dimension. This composition allows differing number of nodes and edges over examples in one batch:

- edge_index와 edge_attr을 통해 sparse한 block diagonal adjacency matrices를 만든다. 
- 이 구성은 노드와 엣지수가 다른 example들이 한 배치에 있는 것을 가능하게 해준다.

PyTorch Geometric contains its own '''torch_geometric.data.DataLoader''', which already takes care of this concatenation process. Let’s learn about it in an example:

In [38]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [41]:
for batch in loader:
    batch
    print(batch)

Batch(batch=[1042], edge_index=[2, 4038], x=[1042, 21], y=[32])
Batch(batch=[1170], edge_index=[2, 4368], x=[1170, 21], y=[32])
Batch(batch=[1027], edge_index=[2, 4070], x=[1027, 21], y=[32])
Batch(batch=[977], edge_index=[2, 3790], x=[977, 21], y=[32])
Batch(batch=[1058], edge_index=[2, 3994], x=[1058, 21], y=[32])
Batch(batch=[1038], edge_index=[2, 4014], x=[1038, 21], y=[32])
Batch(batch=[963], edge_index=[2, 3726], x=[963, 21], y=[32])
Batch(batch=[916], edge_index=[2, 3630], x=[916, 21], y=[32])
Batch(batch=[1084], edge_index=[2, 3702], x=[1084, 21], y=[32])
Batch(batch=[1068], edge_index=[2, 3856], x=[1068, 21], y=[32])
Batch(batch=[1013], edge_index=[2, 3862], x=[1013, 21], y=[32])
Batch(batch=[1136], edge_index=[2, 4350], x=[1136, 21], y=[32])
Batch(batch=[983], edge_index=[2, 3854], x=[983, 21], y=[32])
Batch(batch=[1017], edge_index=[2, 3940], x=[1017, 21], y=[32])
Batch(batch=[1139], edge_index=[2, 4346], x=[1139, 21], y=[32])
Batch(batch=[1180], edge_index=[2, 4368], x=[118

'''torch_geometric.data.Batch''' inherits from '''torch_geometric.data.Data''' and contains an additional attribute: '''batch'''.

'''batch''' is a column vector which maps each node to its respective graph in the batch:

You can use it to, e.g., '''average node features''' in the node dimension for each graph individually:

In [42]:
from torch_scatter import scatter_mean
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [43]:
for data in loader:
    x = scatter_mean(data.x, data.batch, dim=0)
    print(x.size())
    

torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([24, 21])


# Data Transforms

Transforms are a common way in '''torchvision''' to transform images and perform augmentation. PyTorch Geometric comes with its own transforms, which expect a '''Data''' object as input and return a new transformed '''Data''' object. Transforms can be chained together using '''torch_geometric.transforms.Compose''' and are applied before saving a processed dataset on disk ('''pre_transform''') or before accessing a graph in a dataset ('''transform''').

- torchvision말고 별도의 transform이 가능하다.
- Data obj --> Data obj
- pre_transform으로 지정
- torch_geometric.transforms

Let’s look at an example, where we apply transforms on the ShapeNet dataset (containing 17,000 3D shape point clouds and per point labels from 16 shape categories).

In [47]:
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'])

dataset[0]

Data(category=[1], pos=[2518, 3], x=[2518, 3], y=[2518])

We can convert the point cloud dataset into a graph dataset by generating nearest neighbor graphs from the point clouds via transforms:

- point cloud dataset --> a graph dataset(KNN graphs viia transforms)

In [52]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6))

dataset[0]

Data(category=[1], pos=[2518, 3], x=[2518, 3], y=[2518])

# Note

We use the '''pre_transform''' to convert the data before saving it to disk (leading to faster loading times). Note that the next time the dataset is initialized it will already contain graph edges, even if you do not pass any transform.

In addition, we can use the transform argument to '''randomly augment''' a Data object, e.g. translating each node position by a small number:

In [53]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6),
                    transform=T.RandomTranslate(0.01))

dataset[0]

Data(category=[1], pos=[2518, 3], x=[2518, 3], y=[2518])

You can find a complete list of all implemented transforms at '''torch_geometric.transforms'''.

# Learning Methods on Graphs

After learning about data handling, datasets, loader and transforms in PyTorch Geometric, it’s time to implement our first graph neural network!

We will use a simple GCN layer and replicate the experiments on the Cora citation dataset. For a high-level explanation on GCN, have a look at its blog post.

We first need to load the Cora dataset:

In [55]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
dataset

Cora()

Note that we do not need to use transforms or a dataloader. Now let’s implement a two-layer GCN:

In [56]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

The constructor defines two '''GCNConv''' layers which get called in the forward pass of our network. Note that the non-linearity is not integrated in the conv calls and hence needs to be applied afterwards (something which is consistent accross all operators in PyTorch Geometric). Here, we chose to use ReLU as our intermediate non-linearity between and finally output a softmax distribution over the number of classes. Let’s train this model on the train nodes for 200 epochs:

- forward path로 2 layer의 GCNCConv를 만들었다. 
- conv에는 비선형성이 존재하지않음으로 따로 만들어주었다. 
- ReLU, softmax를 선택하였다. 

In [57]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [58]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8070


That is all it takes to implement your first graph neural network. The easiest way to learn more about graph convolution and pooling is to study the examples in the '''examples/''' directory and to browse '''torch_geometric.nn'''. Happy hacking!